In [1]:
# load libraries
import xarray as xr
import numpy as np

from argopy import DataFetcher as ArgoDataFetcher

from datetime import datetime, timedelta
import pandas as pd

# User defined functions:
def get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_in,time_f):
    
    """Function to get argo data for a given lat,lon box (using Argopy), 
       and return a 2D array collection of vertical profile for the given region
    
    Parameters
    ----------
    llon : int
        left longitude
    rlon : int
        right longtidue
    ulat : int
        upper latitude
    llat : int
        lower latitude
    time_in : str/datetime object
        the start time of desired range, formatted Y-m-d
    time_f : str/datetime object
        the end time of desired range, formatted Y-m-d
    
    
    Returns
    ---------
    xarray
        The result is a xarray of the vertical profile for the given range and region.
        
    
    """

    ds_points = ArgoDataFetcher(src='erddap').region([llon,rlon, llat,ulat, depthmin, depthmax,time_in,time_f]).to_xarray()
    ds_profiles = ds_points.argo.point2profile()
    return ds_profiles

def spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end):
    """Function that gets the argo data for given latitude and longitude bounding box
       (using Argopy), and given start and end time range to return a 2D array collection of vertical
       profile for the given region and time frame
    
    Parameters
    ----------
    llon : int
        left longitude
    rlon : int
        right longtidue
    ulat : int
        upper latitude
    llat : int
        lower latitude
    time_in : str/datetime object
        the start time of desired range, formatted Y-m-d
    time_f : str/datetime object
        the end time of desired range, formatted Y-m-d
    
    
    Returns
    ---------
    xarray
        The result is a xarray of the vertical profile for the given range and region.
        
    
    """

    
    #step
    max_dt = timedelta(days = 10)
    
    if isinstance(time_start, str):
        time_start = datetime.strptime(time_start,"%Y-%m-%d")
    if isinstance(time_end, str):
        time_end = datetime.strptime(time_end,"%Y-%m-%d")
        
    if time_end - time_start <= max_dt:
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)
        return ds
    else:
        early_end = time_start+max_dt
        ds = get_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,early_end)
        print("Retrived data from " + str(time_start) + " to " + str(early_end) + ", retreived " + str(len(ds.N_PROF)) + " profiles")
        ds2 = spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax, early_end,time_end)
        return xr.concat([ds,ds2],dim='N_PROF')


In [9]:
llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=1400
time_start='2014-01-01'
time_end='2020-01-01'
ds=spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)

Retrived data from 2014-01-01 00:00:00 to 2014-01-11 00:00:00, retreived 525 profiles
Retrived data from 2014-01-11 00:00:00 to 2014-01-21 00:00:00, retreived 532 profiles
Retrived data from 2014-01-21 00:00:00 to 2014-01-31 00:00:00, retreived 517 profiles
Retrived data from 2014-01-31 00:00:00 to 2014-02-10 00:00:00, retreived 533 profiles
Retrived data from 2014-02-10 00:00:00 to 2014-02-20 00:00:00, retreived 552 profiles
Retrived data from 2014-02-20 00:00:00 to 2014-03-02 00:00:00, retreived 537 profiles
Retrived data from 2014-03-02 00:00:00 to 2014-03-12 00:00:00, retreived 544 profiles
Retrived data from 2014-03-12 00:00:00 to 2014-03-22 00:00:00, retreived 545 profiles
Retrived data from 2014-03-22 00:00:00 to 2014-04-01 00:00:00, retreived 548 profiles
Retrived data from 2014-04-01 00:00:00 to 2014-04-11 00:00:00, retreived 534 profiles
Retrived data from 2014-04-11 00:00:00 to 2014-04-21 00:00:00, retreived 538 profiles
Retrived data from 2014-04-21 00:00:00 to 2014-05-01 0

ValueError: Cannot seek strteaming HTTP file

In [14]:
ds

<xarray.Dataset>
Dimensions:          (N_LEVELS: 999, N_PROF: 3141)
Coordinates:
  * N_LEVELS         (N_LEVELS) int64 0 1 2 3 4 5 6 ... 993 994 995 996 997 998
    LONGITUDE        (N_PROF) float64 -20.17 -38.55 -9.736 ... -11.7 -57.31
    LATITUDE         (N_PROF) float64 50.34 27.32 32.99 ... 33.91 45.06 34.52
    TIME             (N_PROF) datetime64[ns] 2014-01-01T00:32:00 ... 2014-02-...
  * N_PROF           (N_PROF) int64 91 361 389 314 346 317 ... 414 200 8 359 263
Data variables:
    CYCLE_NUMBER     (N_PROF) int64 170 106 93 96 119 201 ... 12 52 29 96 62 214
    DATA_MODE        (N_PROF) <U1 'D' 'D' 'R' 'D' 'D' ... 'D' 'A' 'D' 'D' 'A'
    DIRECTION        (N_PROF) <U1 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    PLATFORM_NUMBER  (N_PROF) int64 4901128 6900773 6900957 ... 6901021 6900112
    POSITION_QC      (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    PRES             (N_PROF, N_LEVELS) float32 6.5 11.0 20.6 ... nan nan nan
    PRES_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    PSAL             (N_PROF, N_LEVELS) float32 35.36283 35.36283 ... nan nan
    PSAL_QC          (N_PROF, N_LEVELS) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 nan
    TEMP             (N_PROF, N_LEVELS) float32 11.07 11.075 11.079 ... nan nan
    TEMP_QC          (N_PROF, N_LEVELS) float64 1.0 1.0 1.0 1.0 ... 1.0 1.0 nan
    TIME_QC          (N_PROF) int64 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
Attributes:
    DATA_ID:              ARGO
    DOI:                  http://doi.org/10.17882/42182
    Fetched_from:         https://www.ifremer.fr/erddap
    Fetched_by:           jovyan
    Fetched_date:         2020/08/14
    Fetched_constraints:  phy_[x=-90.00/0.00; y=0.00/70.00; z=0.0/1400.0; t=2...
    Fetched_uri:          https://www.ifremer.fr/erddap/tabledap/ArgoFloats.n...
    history:              Variables filtered according to DATA_MODE; Variable...

In [13]:
ds.to_netcdf(str('/home/jovyan/ohw20-proj-pyxpcm/data/2014-Jan_to_oct.nc'))

In [ ]:
llon=-90;rlon=0
ulat=70;llat=0 
depthmin=0;depthmax=1400
time_start='2014-11-01'
time_end='2020-01-01'
ds=spliced_argo_region_data(llon,rlon,llat,ulat,depthmin,depthmax,time_start,time_end)

Retrived data from 2014-11-01 00:00:00 to 2014-11-11 00:00:00, retreived 622 profiles
Retrived data from 2014-11-11 00:00:00 to 2014-11-21 00:00:00, retreived 585 profiles
Retrived data from 2014-11-21 00:00:00 to 2014-12-01 00:00:00, retreived 604 profiles
Retrived data from 2014-12-01 00:00:00 to 2014-12-11 00:00:00, retreived 582 profiles
Retrived data from 2014-12-11 00:00:00 to 2014-12-21 00:00:00, retreived 575 profiles
Retrived data from 2014-12-21 00:00:00 to 2014-12-31 00:00:00, retreived 573 profiles
Retrived data from 2014-12-31 00:00:00 to 2015-01-10 00:00:00, retreived 572 profiles
Retrived data from 2015-01-10 00:00:00 to 2015-01-20 00:00:00, retreived 568 profiles
Retrived data from 2015-01-20 00:00:00 to 2015-01-30 00:00:00, retreived 569 profiles
Retrived data from 2015-01-30 00:00:00 to 2015-02-09 00:00:00, retreived 581 profiles
Retrived data from 2015-02-09 00:00:00 to 2015-02-19 00:00:00, retreived 582 profiles
Retrived data from 2015-02-19 00:00:00 to 2015-03-01 0